# Step 2: Model Building & Evaluation
Using the training and test data sets we constructed in the `Code/1_data_ingestion_and_preparation.ipynb` Jupyter notebook, this notebook builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict failure in aircraft engines. We will store the model for deployment in an Azure web service which we build in the `Code/3_operationalization.ipynb` Jupyter notebook.

In [225]:

# import the libraries
import os
import pandas as pd
import numpy as np

from azureml.core import  (Workspace,Run,VERSION, 
                           Experiment,Datastore)

from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score

## Azure ML workspace

In [226]:
subscription_id = 'fe375bc2-9f1a-4909-ad0d-9319806d5e97'
resource_group = 'vienna_rg'
workspace_name = 'viennadocs'
location = 'eastus2'

In [230]:
project_folder = os.getcwd()
exp_name = "deep_predictive_maintenance"

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

ws.write_config()
print('Workspace loaded:', ws.name)

Wrote the config file config.json to: /home/sasuke/dev/amlsamples/deep_predictive_maintenance/aml_config/config.json
Workspace loaded: viennadocs


## Load feature data set

We have previously created the labeled data set in the `Code\1_Data Ingestion and Preparation.ipynb` Jupyter notebook and stored it in default datastore of the AML workspace.

Here We download the training/testing datasets here.

In [231]:
data_dir = os.path.join(project_folder, 'data')

ds = Datastore.get(ws,'workspaceblobstore')
ds.download(data_dir, overwrite=True, show_progress = True)
print(os.listdir(data_dir))

['preprocessed_test_file.csv', 'data', 'preprocessed_train_file.csv']


Load the data and dump a short summary of the resulting DataFrame.

In [216]:
train_df = pd.read_csv(os.path.join(data_dir, 'preprocessed_train_file.csv'))
train_df.head(5)

,engine_id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [217]:
test_df = pd.read_csv(os.path.join(data_dir, 'preprocessed_test_file.csv'))
test_df.head(5)

,engine_id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138,0,0


## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering, the manual construction of variable using domain expertise and intuition. This usually makes these models hard to reuse as the feature are specific to the problem scenario and the available data may vary between customers. Perhaps the most attractive advantage of deep learning they automatically do feature engineering from the data, eliminating the need for the manual feature engineering step.

When using LSTMs in the time-series domain, one important parameter is the sequence length, the window to examine for failure signal. This may be viewed as picking a `window_size` (i.e. 5 cycles) for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3). The rolling features included rolling mean and rolling standard deviation over the 5 cycles for each of the 21 sensor values. In deep learning, we allow the LSTMs to extract abstract features out of the sequence of sensor values within the window. The expectation is that patterns within these sensor values will be automatically encoded by the LSTM.

Another critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. Computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing over such a long period. LSTMs are able to use larger window sizes and use all the information in the window as input. 

http://colah.github.io/posts/2015-08-Understanding-LSTMs/ contains more information on the details of LSTM networks.

This notebook illustrates the LSTM approach to binary classification using a sequence_length of 50 cycles to predict the probability of engine failure within 30 days.

In [218]:
# pick a large window size of 50 cycles
sequence_length = 50

We use the [Keras LSTM](https://keras.io/layers/recurrent/) with [Tensorflow](https://tensorflow.org) as a backend. Here layers expect an input in the shape of an array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step.

We define a function to generate this array, as we'll use it repeatedly.

## LSTM Network

Building a Neural Net requires determining the network architecture. In this scenario we will build a network of only 2 layers, with dropout. The first LSTM layer with 100 units, one for each input sequence, followed by another LSTM layer with 50 units. We will also apply dropout each LSTM layer to control overfitting. The final dense output layer employs a sigmoid activation corresponding to the binary classification requirement.

In [224]:
%%writefile lstm.py

import torch 
import torch.nn as nn
import torch.utils.data as utils
from azureml.core import Run

class Network(nn.Module):
    
    def __init__(self, input_size=25, hidden_size=16, nb_layers=1, dropout=.5, nb_classes=2):
        super(Network, self).__init__()
        self.hidden_size = hidden_size
        self.nb_layers = nb_layers
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size, hidden_size, nb_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, nb_classes)
    
    def forward(self, x):
        
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.nb_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.nb_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # retrieve hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

def train(dataloader, input_size,hidden_size, nb_layers, dropout, nb_classes):
    
    run = Run.get_context()
    
    network = Network(input_size,hidden_size, nb_layers, dropout, nb_classes).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(nb_epochs):
        for i, (X, y) in enumerate(dataloader):
            X = X.reshape(-1, sequence_length, input_size).to(device)
            y = y.to(device)

            # Forward pass
            y_pred = network(X)
            loss = criterion(y_pred, y)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            y_pred_np = y_pred.to('cpu').data.numpy()
            y_pred_np = np.argmax(y_pred_np, axis=1)

            y_np = y.data.to('cpu').data.numpy()
            accuracy = accuracy_score(y_np, y_pred_np)

            if (i+1) % 100 == 0:
                print ('epoch [{}/{}], loss: {:.4f}, accuracy: {:.2f}'
                                   .format(epoch+1,nb_epochs, loss.item(), accuracy)) 
                run.log('loss', loss.item())
                run.log('accuracy', accuracy)
                
if __name__ == '__main__':
    

    input_size = 25
    hidden_size = 16
    nb_layers = 1
    nb_classes = 2
    batch_size = 64
    nb_epochs = 2
    learning_rate = .001
    dropout = .5

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    X_train = torch.from_numpy(seq_array)
    y_train = torch.from_numpy(label_array)#.type(torch.FloatTensor)

    print(X_train.size(), y_train.size())

    dataset = utils.TensorDataset(X_train,y_train) 
    dataloader = utils.DataLoader(dataset)
    train(dataloader, input_size,hidden_size, nb_layers, dropout, nb_classes)
    

Writing lstm.py


In [ ]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, 
                      script='lstm.py', 
                      run_config=conda_run_config, 
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--data-folder', str(ds.as_download())] 
                     ) 
run = exp.submit(config=src)

Since we have many more healthy cycles than failure cycles, we also look at precision and recall. In all cases, we assume the model threshold is at $Pr = 0.5$. In order to tune this, we need to look at a test data set. 

In [15]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = 2 * (precision * recall) / (precision + recall)
print( 'Training Precision: ', precision, '\n', 'Training Recall: ', recall, '\n', 'Training F1 Score:', f1)
run_logger.log("Training Precision", precision)
run_logger.log("Training Recall", recall)
run_logger.log("Training F1 Score", f1)

Training precision 	=  0.9709281961471103 
Training recall 	=  0.8941935483870967


## Model testing
Next, we look at the performance on the test data. Only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [162]:
seq_array_test_last = [test_df[test_df['engine_id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['engine_id'].unique() \
                               if len(test_df[test_df['engine_id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(93, 50, 25)

We also ned the test set labels in the correct format.

In [165]:
y_mask = [len(test_df[test_df['engine_id']==id]) >= sequence_length for id in test_df['engine_id'].unique()]

label_array_test_last = test_df.groupby('engine_id')['label1'].nth(-1)[y_mask].values
#label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last = label_array_test_last.reshape(-1)

print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(93, 50, 25)
(93,)


Now we can test the model with the test data. We report the model accuracy on the test set, and compare it to the training accuracy. By definition, the training accuracy should be optimistic since the model was optimized for those observations. The test set accuracy is more general, and simulates how the model was intended to be used to predict forward in time. This is the number we should use for reporting how the model performs.

In [173]:
# test metrics

X_test = torch.from_numpy(seq_array_test_last).to(device)
y_test = torch.from_numpy(label_array_test_last)
y_pred = network(X_test)

y_pred_np = y_pred.to('cpu').data.numpy()
y_test_np = y_test.data.numpy()

y_pred_np = np.argmax(y_pred_np, axis=1)
accuracy = accuracy_score(y_test_np, y_pred_np)

print('Test Accurracy: {}'.format(accuracy))


Test Accurracy: 0.8817204301075269


Similarly for the test set confusion matrix. 

In [172]:

print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_test_np, y_pred_np)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[67,  1],
       [10, 15]])

The confusion matrix uses absolute counts, so comparing the test and training set confusion matrices is difficult. Instead, it is  better to use precision and recall. 

 * _Precision_ measures how accurate your model predicts failures. What percentage of the failure predictions are actually failures.
 * _Recall_ measures how well the model captures thos failures. What percentage of the true failures did your model capture.
 
These measures are tightly coupled, and you can typically only choose to maximize one of them (by manipulating the probability threshold) and have to accept the other as is.


In [175]:
# compute precision and recall
precision_test = precision_score(y_test_np, y_pred_np)
recall_test = recall_score(y_test_np, y_pred_np)
f1_test = 2 * (y_test_np* y_pred_np)/ (y_test_np+ y_pred_np)
print( 'Test Precision: ', precision_test, '\n', 'Test Recall: ', recall_test, '\n', 'Test F1 Score:', f1_test)


Test Precision:  0.9375 
 Test Recall:  0.6 
 Test F1 Score: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan  0. nan  1. nan
 nan  0. nan nan nan nan nan  1. nan nan  1.  1.  1.  0. nan  0.  0.  1.
 nan nan nan nan nan nan  1. nan nan  0.  0. nan nan  1. nan nan nan nan
  0. nan nan  0. nan  1. nan  1. nan nan nan nan nan nan nan  1. nan nan
 nan nan  1.  1. nan nan nan nan nan nan  0.  0.  1. nan nan nan nan nan
 nan nan  1.]


/anaconda/envs/amlenv/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


## Saving the model  

The LSTM network is made up of two components, the architecture and the model weights. We'll save these model components in two files, the architecture in a `json` file that the `keras` package can use to rebuild the model, and the weights in an `HDF5` heirachy that rebuild the exact model. 

In [198]:
model_name = 'lstm.pth'
model_path = os.path.join(os.getcwd(), model_name)
torch.save(network.state_dict(), model_path)
print("Model saved")

Model saved


To test the save operations, we can reload the model files into a test model `loaded_model` and rescore the test dataset.

In [199]:
the_model = Network()
the_model.load_state_dict(torch.load(model_path))
print(the_model)

Network(
  (dropout): Dropout(p=0.5)
  (lstm): LSTM(25, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=2, bias=True)
)


# Persist the model

In order to pass the model to our next notebook, we will write the model files to the shared folder within the Azure ML Workbench project. https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

In the `Code\3_operationalization.ipynb` Jupyter notebook, we will create the functions needed to operationalize and deploy any model to get realtime predictions. The artifacts created will be stored in one of your Azure storage containers for you to deploy and test your own web service.

In [207]:
run = Run.get_context()
run.register_model(model_name = model_name, model_path = model_name)

RunEnvironmentException: Failed to load a submitted run, if outside of an execution context, use project.start_run to initialize an azureml.core.Run.